<a href="https://colab.research.google.com/github/ykitaguchi77/Kaggle/blob/main/Bunkruptcy-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**KaggleのデータセットをGoogle Driveにダウンロードする**

## Google Driveのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## データセットの準備

In [3]:
# kaggle ライブラリのインストール
!pip install kaggle

# 一時フォルダに .kaggleフォルダを作成
!mkdir ~/.kaggle

# MyDrive の kaggle.json を一時フォルダ内の .kaggleフォルダにコピー
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/

# アクセス権限の設定
!chmod 600 ~/.kaggle/kaggle.json

!mkdir ~/.kaggle

# zipファイルのダウンロード
!kaggle competitions download -c bankruptcy-risk-prediction -p /content
# 解凍
!unzip /content/bankruptcy-risk-prediction.zip -d /content

#Zipフォルダを削除
!rm /content/bankruptcy-risk-prediction.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
mkdir: cannot create directory ‘/root/.kaggle’: File exists
bankruptcy-risk-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/bankruptcy-risk-prediction.zip
  inflating: /content/submission_example.csv  
  inflating: /content/test.csv       
  inflating: /content/train.csv      


In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


#import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

!pip install optuna
import optuna
import optuna.integration.lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')